In [127]:
import gradio as gr
import pandas as pd
import numpy as np
from pathlib import Path

data_array = [
    {"path": "projectReinforcementLearning/images/ap_1.jpg", "category": "category1"},
    {"path": "projectReinforcementLearning/images/ap_6.jpg", "category": "category2"},
    # Add more image paths and categories as needed
]
# Initial index for displaying the first image
current_index = 0
# Function to update the displayed image and category
def update_data():
    global current_index
    current_index = (current_index + 1) % len(data_array)
    return data_array[current_index]["path"]#, data_array[current_index]["category"]
    
def save_rating(radio, slider):
    
    data = [{
        "category" : radio,
        "level_of_mistake": slider
    }]
    
    df = pd.DataFrame(data)
    print(df)
    
    path = Path("test_csv.csv")
    if path.is_file():
        df.to_csv(path, mode='a', header=False, index=False)
    else:
        df.to_csv(path, mode='a', header=True, index=False)
    
def display_categories(radio):

    categories_radio = gr.Radio(["Alignement Problems", "Incorrect proportions", "Number of features", "Wrong Aspects", "unrealistic"], interactive=True, visible=True) 
    slider = gr.Slider(minimum=0, maximum=5, step=1, interactive=True, visible=True)
    outputs = [categories_radio, slider]
    return outputs
  
with gr.Blocks() as main:
    gr.Markdown(
        """
        # Main study
        
        """)
    with gr.Column():
        
        with gr.Row():
            image = gr.ImageEditor("projectReinforcementLearning/images/ap_1.jpg")
        
            
            with gr.Column():
                gr.Markdown(
                    """
                    # Do you spot any Mistake in this picture?
                    """
                )
                yes_button = gr.Button("YES")
                # yes_button.click(display_categories, inputs=[], outputs=[gr.Radio(), gr.Slider()])
                no_button  = gr.Button("NO")
                #radio = gr.Radio(["YES", "NO"], label="Select")   
                
                
                
                categories_radio = gr.Radio(visible=False)
                
                slider = gr.Slider(visible=False)

                yes_button.click(display_categories, 
                     inputs=[],
                     outputs=[categories_radio, slider]
                )
                

                submit_button = gr.Button("Submit")
    
        gr.Textbox(label="Promt", value="this is the promt")
             
            
    
    next = gr.Button("Next")
    next.click()
    
    
    no_button.click(fn=update_data, inputs=[], outputs=[image])

    submit_button.click(
        fn=save_rating,
        inputs=[categories_radio, slider],
        outputs=[]
    )
    


main.launch(inbrowser=True)

C:\Users\kohnm\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\utils.py:840: UserWarning: Expected 1 arguments for function <function display_categories at 0x0000029EF278B520>, received 0.
  warnings.warn(
C:\Users\kohnm\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\utils.py:844: UserWarning: Expected at least 1 arguments for function <function display_categories at 0x0000029EF278B520>, received 0.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7954

To create a public link, set `share=True` in `launch()`.


C:\Users\kohnm\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\helpers.py:837: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


             category  level_of_mistake
0  Number of features                 0
             category  level_of_mistake
0  Number of features                 0


In [138]:
import gradio as gr
import pandas as pd
import numpy as np
from pathlib import Path

data_array = [
    {"path": "projectReinforcementLearning/images/ap_1.jpg", "category": "category1"},
    {"path": "projectReinforcementLearning/images/ap_6.jpg", "category": "category2"},
    # Add more image paths and categories as needed
]
# Initial index for displaying the first image
current_index = 0
# Function to update the displayed image and category
def update_data():
    global current_index
    current_index = (current_index + 1) % len(data_array)
    return data_array[current_index]["path"]#, data_array[current_index]["category"]
    
def save_rating(radio, slider):
    
    data = [{
        "category" : radio,
        "level_of_mistake": slider
    }]
    
    df = pd.DataFrame(data)
    print(df)
    
    path = Path("test_csv.csv")
    if path.is_file():
        df.to_csv(path, mode='a', header=False, index=False)
    else:
        df.to_csv(path, mode='a', header=True, index=False)
    
def display_categories(radio):

    categories_radio = gr.Radio(["Alignement Problems", "Incorrect proportions", "Number of features", "Wrong Aspects", "unrealistic"], interactive=True, visible=True) 
    slider = gr.Slider(minimum=0, maximum=5, step=1, interactive=True, visible=True)
    outputs = [categories_radio, slider]
    return outputs
  
with gr.Blocks() as main:
    with gr.Tab("Introduction"):
        gr.Markdown(
            """
            # Main study
            
            """)
        with gr.Column():
            
            with gr.Row():
                image = gr.ImageEditor("projectReinforcementLearning/images/ap_1.jpg")
            
                
                with gr.Column():
                    gr.Markdown(
                        """
                        # Do you spot any Mistake in this picture?
                        """
                    )
                    yes_button = gr.Button("YES")
                    # yes_button.click(display_categories, inputs=[], outputs=[gr.Radio(), gr.Slider()])
                    no_button  = gr.Button("NO")
                    #radio = gr.Radio(["YES", "NO"], label="Select")   
                    
                    
                    
                    categories_radio = gr.Radio(visible=False)
                    
                    slider = gr.Slider(visible=False)
    
                    yes_button.click(display_categories, 
                         inputs=[],
                         outputs=[categories_radio, slider]
                    )
                    
    
                    submit_button = gr.Button("Submit")
        
            gr.Textbox(label="Promt", value="this is the promt")
                 
                
        
        next = gr.Button("Next")
        next.click()
        
        
        no_button.click(fn=update_data, inputs=[], outputs=[image])
    
        submit_button.click(
            fn=save_rating,
            inputs=[categories_radio, slider],
            outputs=[]
        )
    
    with gr.Tab(label="Introduction"):
        gr.Button("Test")
    


main.launch(inbrowser=True)

C:\Users\kohnm\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\utils.py:840: UserWarning: Expected 1 arguments for function <function display_categories at 0x0000029EF53CF400>, received 0.
  warnings.warn(
C:\Users\kohnm\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\utils.py:844: UserWarning: Expected at least 1 arguments for function <function display_categories at 0x0000029EF53CF400>, received 0.
  warnings.warn(


TypeError: Tab.__init__() got an unexpected keyword argument 'interactive'

In [139]:
pip list

Package                   Version
------------------------- ---------------
aiofiles                  23.2.1
altair                    5.2.0
annotated-types           0.6.0
anyio                     4.2.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     23.2.0
Babel                     2.14.0
beautifulsoup4            4.12.2
bleach                    6.1.0
certifi                   2023.11.17
cffi                      1.16.0
charset-normalizer        3.3.2
click                     8.1.7
colorama                  0.4.6
comm                      0.2.1
contourpy                 1.2.0
cycler                    0.12.1
debugpy                   1.8.0
decorator                 5.1.1
defusedxml                0.7.1
exceptiongroup            1.2.0
executing                 2.0.1
facenet-pytorch           2.5.3
fastapi                   0.109.0
fastjsonschem